In [ ]:
import os
import math
import numpy as np
import pandas as pd
import random
import cv2
import matplotlib.pyplot as plt
import scipy as sp
from scipy import signal
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Reshape, Dense, Conv2D, MaxPool2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam

np.set_printoptions(suppress=True, precision=2)

LABEL_PATH = '../input/culane/driver_161_90frame_labels/'
IMG_PATH = '../input/culane/driver_161_90frame/'

all_labels = []
for path, subdirs, files in os.walk(LABEL_PATH):
    for name in files:
        all_labels.append(os.path.join(path, name))
all_labels.sort()

all_images = []
for path, subdirs, files in os.walk(IMG_PATH):
    for name in files:
        all_images.append(os.path.join(path, name))
all_images.sort()
for idx, file in enumerate(all_images):
    if file.split('.')[-1] == 'txt':
        all_images.remove(file)
        
#Helper function: Round up to ten
def roundup(x):
    return int(math.ceil(x / 10.0)) * 10
        
def derivative_xy(img):  
    #prewire
    H_x = np.array([[1,1, 1,-1,-1,-1]])
    H_y = np.array([[1,1,1],[-1,-1,-1]])
    
    derv_x = sp.signal.convolve2d(img, H_x, mode='same')
    derv_y = sp.signal.convolve2d(img, H_y, mode='same')

    return derv_x, derv_y

#Arctan function
def get_angles(derv_x, derv_y):
    #inverse tan (derv_x / derv_y)
    return np.arctan2(derv_x, derv_y) #one from the lecture

#returns position of first nonzero value
def pos_first_nonzero(data):
    a = data[-1][:]
    for i in range(a.shape[0]):
        if a[i] > 0:
            return i

print(len(all_images))

def applyImageMask(img):
    # Getting image height and width
    height = img.shape[0]
    width = img.shape[1]

    # Creating a triangle polygon to represent valid image pixels (the ones we are interested in)
    tri_poly = np.array([
                            [
                                (0, height),
                                (width // 2, height // 2), 
                                (width, height)
                            ]
                        ])

    lane_mask = np.zeros((height, width))
    lane_mask = np.uint8(lane_mask)
    cv2.fillPoly(lane_mask, tri_poly, 255)

    # Getting the part of the image we are interested in
    masked_img = cv2.bitwise_and(img, lane_mask)
    return masked_img

In [ ]:

number_of_photos = 60 ########################################################################################NUMBER OF IMAGES
counter = 0

imagesBeingTrained = []


#Declaring Empty Arrays
leftLineAngle = np.zeros(number_of_photos)
rightLineAngle = np.zeros(number_of_photos)

leftLineXCoord = np.zeros(number_of_photos)
rightLineXCoord = np.zeros(number_of_photos)

images = np.zeros((number_of_photos, 590, 1640))

print("LeftLineShape: ", leftLineAngle.shape)
print("Image: ", images.shape)


# rand_index = np.random.randint(0, len(all_labels), 1)[0]

randomImageIndices = random.sample(range(1, len(all_labels)), number_of_photos)
# for i in range(number_of_photos):
for i in randomImageIndices:
    rand_index = i
    label = cv2.imread(all_labels[rand_index])
    img = cv2.cvtColor(cv2.imread(all_images[rand_index]), cv2.COLOR_BGR2GRAY) #GRAYSCALE
    
    #Applying Image mask
#     img = applyImageMask(img)
    
    #added to the list of images being trained
    imagesBeingTrained.append(i)
    
    sample_masks = []
    sample_mask2 = np.all(label==(2,2,2), axis=-1)*255
    sample_mask3 = np.all(label==(3,3,3), axis=-1)*255

    sample_masks.append(sample_mask2)
    sample_masks.append(sample_mask3)

######################################
#     fig, axs = plt.subplots(3, figsize=(30, 30)) #for graphing

#     axs[0].imshow(img)
#     axs[0].axis('off')

#     axs[1].imshow(sample_mask2)
#     axs[1].axis('on')

#     axs[2].imshow(sample_mask3)
#     axs[2].axis('on')
######################################


    #get derivatives
    derv_x1, derv_y1 = derivative_xy(sample_mask2)
    derv_x2, derv_y2 = derivative_xy(sample_mask3)

    #Get angles
    angles1 = get_angles(derv_x1, derv_y1)
    angles2 = get_angles(derv_x2, derv_y2)

    pos1 = pos_first_nonzero(angles1)
    pos2 = pos_first_nonzero(angles2)

    #convert from rad to degrees
    angles1 = np.degrees(angles1) #to degrees
    angles2 = np.degrees(angles2) #to degrees

    a1 = np.floor(np.average(np.nonzero(angles1)))
    a2 = np.floor(np.average(np.nonzero(angles2)))
    
    #Lane line 1 is under 90 deg, the other is over 90
    while(a1 > 90): 
        a1-=90
    while(a2 > 180): 
        a2-=180

    #Add Data to the matrix
    leftLineAngle[counter] = a1
    rightLineAngle[counter] = a2
    
    leftLineXCoord[counter] = pos1
    rightLineXCoord[counter] = pos2
    
    images[counter] = img
    counter+=1

#Change Nan's to zero's
leftLineAngle = np.nan_to_num(leftLineAngle)


print("Xtrain shape:")
print(images.shape)
print("Ytrain shape:")
print(leftLineAngle.shape) #left lane line

In [ ]:
#Keras Model: Simple Conv2d NN
model = Sequential([
    Input((590, 1640)),
    Reshape((590, 1640, 1)),
    Conv2D(32, kernel_size=(3, 3), padding='valid'),
    MaxPool2D(pool_size=(2,2), strides=(2,2)),
    Reshape((294 * 819 * 32,)),
    Dense(1, activation='relu')                   
])
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])

model.summary()

In [ ]:
#TRAIN model
epoc = 10
history1 = model.fit(images, leftLineAngle, validation_split = 0.1, epochs=epoc)

In [ ]:
# Plot the training accuracy and cross validation accuracy
# Make two separate plots

# Y values
acc_val = history1.history['acc']
val_acc_val = history1.history['val_acc']



# x values
x_axis = np.linspace(0, epoc, epoc)

plt.figure(figsize=(epoc, 6))
plt.subplot(2,1,1)
plt.plot(x_axis, acc_val);

plt.subplot(2,1,2)
plt.plot(x_axis, val_acc_val);

In [ ]:
    imageNumber = 65
    while (imageNumber not in imagesBeingTrained):
        imageNumber = random.randint(1, len(all_labels)-1)
    
    label = cv2.imread(all_labels[imageNumber])
#     img = cv2.cvtColor(cv2.imread(all_images[imageNumber]), cv2.COLOR_BGR2RGB) #GRAYSCALE
    img = cv2.cvtColor(cv2.imread(all_images[imageNumber]), cv2.COLOR_BGR2GRAY) #GRAYSCALE

    # Loading road image and making a grayscale version
#     img_road_rgb = cv2.cvtColor(cv2.imread(all_images[imageNumber]), cv2.COLOR_BGR2RGB)
#     img_road_rgb_final = cv2.cvtColor(img_road_bgr, cv2.COLOR_BGR2RGB)

    leftLine = np.all(label==(2,2,2), axis=-1)*255
    rightLine = np.all(label==(3,3,3), axis=-1)*255
    
    image = np.array([img])
    
    print(image.shape)
    
#Making a prediction
#     all_preds = model.predict(image)
#     print(all_preds)
    
#     cv2.imshow('Input image',img)
#     cv2.imshow('LeftLane',leftLine)

